In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

## 크롬 드라이버 객체 생성

In [2]:
service = Service(executable_path=ChromeDriverManager().install())

In [3]:
driver = webdriver.Chrome(service=service)

### 웹페이지 로드(여기어때)

In [4]:
# 여기어때
url = 'https://www.yeogi.com/'
driver.get(url)

In [5]:
# 국내 여행지 접근 xpaths
xpaths_place = []
for i in np.arange(1,18):
    xpaths_place.append('//*[@id="__next"]/main/section[10]/article/div[' + str(i) + ']/a')
xpaths_place

['//*[@id="__next"]/main/section[10]/article/div[1]/a',
 '//*[@id="__next"]/main/section[10]/article/div[2]/a',
 '//*[@id="__next"]/main/section[10]/article/div[3]/a',
 '//*[@id="__next"]/main/section[10]/article/div[4]/a',
 '//*[@id="__next"]/main/section[10]/article/div[5]/a',
 '//*[@id="__next"]/main/section[10]/article/div[6]/a',
 '//*[@id="__next"]/main/section[10]/article/div[7]/a',
 '//*[@id="__next"]/main/section[10]/article/div[8]/a',
 '//*[@id="__next"]/main/section[10]/article/div[9]/a',
 '//*[@id="__next"]/main/section[10]/article/div[10]/a',
 '//*[@id="__next"]/main/section[10]/article/div[11]/a',
 '//*[@id="__next"]/main/section[10]/article/div[12]/a',
 '//*[@id="__next"]/main/section[10]/article/div[13]/a',
 '//*[@id="__next"]/main/section[10]/article/div[14]/a',
 '//*[@id="__next"]/main/section[10]/article/div[15]/a',
 '//*[@id="__next"]/main/section[10]/article/div[16]/a',
 '//*[@id="__next"]/main/section[10]/article/div[17]/a']

# 호텔 & 리조트 버튼 클릭하여 목록 보기

xpath = '//*[@id="category2"]/div/div[2]/div[3]/button'
hotel_click = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
hotel_click.click()

In [6]:
# 각 호텔 들어가는 xpath
xpath_select_hotels = []
for i in np.arange(1,22):
    xpath_select_hotels.append('//*[@id="__next"]/div[1]/main/section/div[2]/a[' + str(i) + ']')
xpath_select_hotels

['//*[@id="__next"]/div[1]/main/section/div[2]/a[1]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[2]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[3]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[4]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[5]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[6]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[7]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[8]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[9]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[10]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[11]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[12]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[13]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[14]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[15]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[16]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[17]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/a[18]',
 '//*[@id="__next"]

In [7]:
# 버튼은 1~7(최대) 1번과 7번은 이동화살표, 1,2 번 스킵
# 호텔 리스트 페이지 이동 접근 xpath
xpath_select_button_hotels = []
for i in np.arange(3,8):
    xpath_select_button_hotels.append('//*[@id="__next"]/div[1]/main/section/div[2]/div/div/div/button[' + str(i) + ']')
xpath_select_button_hotels

['//*[@id="__next"]/div[1]/main/section/div[2]/div/div/div/button[3]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/div/div/div/button[4]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/div/div/div/button[5]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/div/div/div/button[6]',
 '//*[@id="__next"]/div[1]/main/section/div[2]/div/div/div/button[7]']

In [8]:
# 호텔 리뷰 버튼 접근 xpath
xpath_select_button_reviews = []
for i in np.arange(3,8):
    xpath_select_button_reviews.append('//*[@id="review"]/div[8]/div/div/button[' + str(i) + ']')
xpath_select_button_reviews

['//*[@id="review"]/div[8]/div/div/button[3]',
 '//*[@id="review"]/div[8]/div/div/button[4]',
 '//*[@id="review"]/div[8]/div/div/button[5]',
 '//*[@id="review"]/div[8]/div/div/button[6]',
 '//*[@id="review"]/div[8]/div/div/button[7]']

### 호텔 데이터 크롤링

In [9]:
# 별점, 작성 시기 추출
def get_stars_date(stars_date_elements):
    lists = []
    star1 = 0
    star2 = 0
    for idx in range(len(stars_date_elements)):
        try:
            star1 = len(stars_date_elements[idx].find_elements(By.CLASS_NAME, 'css-7zs8ep'))
        except NoSuchElementException as e:
            print('1점도 없음')
        try:
            star2 = len(stars_date_elements[idx].find_elements(By.CLASS_NAME, 'css-1um40mz'))
        except NoSuchElementException as e:
            pass
        point = star1 + star2/2
        date = stars_date_elements[idx].text
        lists.append([point, date])
    
    return lists

In [1]:
# 호텔 리뷰 추출, 별점&작성시기 통합
def get_reviews():
    review_dict = []
    button_num=0
    count = 0 # 테스트용 변수

    while(True):
        reviews = driver.find_elements(By.CLASS_NAME,'css-nyr29c')
        stars_date_elements = driver.find_elements(By.CLASS_NAME, 'css-1xow60n')
        
        # 데이터 수집
        review_dict_five = get_stars_date(stars_date_elements)
        
        for idx in range(len(reviews)):
            review_dict_five[idx].append(reviews[idx].text)
            count += 1
    
        # 리뷰 데이터 통합
        review_dict.extend(review_dict_five)    
        
        # 수집 기간 설정 
        if len(review_dict) > 0:
            for idx in range(-5,0):
                if review_dict[idx][1] == '2년 전':
                    del review_dict[idx:]
                    break
            
        # 페이지 이동
        try:
            button_check = driver.find_element(By.XPATH, xpath_select_button_reviews[button_num])
            button_num +=1

            # 테스트용 50개까지
            if count > 5:
                print(count)
                break
            
            if button_check.get_attribute('disabled') is not None: # 버튼 비활성화 > 페이지 종료
                print(f'요소 끝, 최종 댓글 개수 : {count}')
                break
                
            else:
                if button_check.get_attribute('class') == 'css-dv31e9':
                    button_num = 0
                    print('화살표 접근')
                actions = ActionChains(driver).move_to_element(driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/section/div[1]'))
                actions.perform()
                button_check.click()
                driver.implicitly_wait(5)
                
        except NoSuchElementException as e:
            print('접근 오류')
            break
    
    # [[별 점수1, 리뷰내용1],[별 점수2, 리뷰내용2],..]
    return review_dict
    
# get_reviews()

In [2]:
def get_hotel_data(xpath):

    """
    # 호텔 세부사항으로 이동
    # driver.find_element(By.XPATH,'xpath').click()
    # driver.switch_to.window(driver.window_handles[-1])
    """
    
    # 호텔명
    hotel_name = driver.find_element(By.CLASS_NAME,'css-e3nxqe').text
    hotel_dict['호텔명'] = hotel_name
    print(driver.find_element(By.CLASS_NAME,'css-e3nxqe').text)

    # 서비스 및 부대시설
    hotel_service = driver.find_element(By.CLASS_NAME,'css-1vbs9oj').text.split('\n')
    hotel_dict['서비스 및 부대시설'] = hotel_service
    print(driver.find_element(By.CLASS_NAME,'css-1vbs9oj').text.split('\n'))

    # 숙소 소개
    hotel_intro = driver.find_element(By.CLASS_NAME,'css-l22fxz').text
    hotel_dict['숙소 소개'] = hotel_intro
    print(driver.find_element(By.CLASS_NAME,'css-l22fxz').text)
    
    # 판매자 정보 (사업자 번호)
    sell_user = driver.find_element(By.CLASS_NAME,'css-h9kvaf')
    actions = ActionChains(driver).move_to_element(sell_user)
    actions.perform()
    sell_user.click()
    
    hotel_dict['사업자 번호'] = driver.find_elements(By.CLASS_NAME,'css-138kqsc')[-1].text
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-wrapper"]/div/div/header/div/div[1]'))).click()


    """
    # 리뷰 순서 변경
    review_ascending = driver.find_element(By.CLASS_NAME,'css-tfi2u7')
    actions = ActionChains(driver).move_to_element(sell_user)
    actions.perform()
    review_ascending.click()  
    driver.find_element(By.CLASS_NAME,'css-1t2cu3a').click()
    """

    # 리뷰 데이터 추적
    get_reviews()
    
    """
    # 호텔 리스트로 이동
    # driver.close()
    # driver.switch_to.window(driver.window_handles[0])
    """

    # 구현 요구 영역
    # 호텔명 , 서비스 및 부대시설 , 숙소 소개, 판매자 정보, 리뷰정보 통합시키고 return 하기
    return 0

# get_hotel_data()

In [ ]:
# 여기 어때 메인 페이지 기준 탐색 시작
# 지역별 링크
# 데이터 담을 방식 지정 필요함.
for region_path in xpaths_place:
    # 지역 이동(서울, 경기 등..)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, region_path))).click()
    
    # 호텔 & 리조트 버튼 클릭하여 목록 보기
    xpath_select_type = '//*[@id="category2"]/div/div[2]/div[3]/button'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_select_type))).click()

    # 각 호텔 들어가는 xpath
    for select_hotel in xpath_select_hotels: 
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, select_hotel))).click()

        # 리뷰
        get_reviews()

    
    # 여기 어때 메인 페이지로 이동
    driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/header/div[1]/div[1]/a/svg/path').click()
    